In [1]:
# Import likely important packages, etc.
import sys, os, csv, re
from wcmatch import fnmatch
import datetime
import time
import xml.etree.ElementTree as et
from zipfile import ZipFile
import numpy as np
import pandas as pd
import xarray as xr
import shutil

In [2]:
from utils import *

In [3]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Results'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Cal'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/CTDMOG'

In [4]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [6]:
CTDMO = whoi_asset_tracking(excel_spreadsheet,sheet_name,instrument_class='CTDMO',whoi=True,series='G')
CTDMO.head(10)

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
95,CTDMO,G,37-10214,115166,A00103,CGINS-CTDMOG-10214,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00001\n3305-00101-00082\n3305-00101...,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP02HYPM-00002\nGP03FLMA-00004,NaN,NaN,NaN
96,CTDMO,G,37-10215,115167\n118298,A00104,CGINS-CTDMOG-10215,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00002\n3305-00101-00404\n3305-00101...,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
97,CTDMO,G,37-10216,115168,A00105,CGINS-CTDMOG-10216,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00003\n3305-00101-00255\n3305-00101...,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMB-00004,NaN,NaN,NaN
98,CTDMO,G,37-10217,115169\n118299,A00106,CGINS-CTDMOG-10217,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00004\n3305-00101-00399\n3305-00101...,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
99,CTDMO,G,37-10218,115170,A00107,CGINS-CTDMOG-10218,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00005\n3305-00101-00083\n3305-00101...,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00002\nGP03FLMB-00004,NaN,NaN,NaN
100,CTDMO,G,37-10219,115171\n118138,A00108,CGINS-CTDMOG-10219,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00006\n3305-00101-00436\n3305-00101...,NaN,NaN,3305-00900-00152\n3305-00900-00392,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00001\nGP03FLMA-00003\nGI02HYPM-00004,NaN,NaN,NaN
101,CTDMO,G,37-10220,115172\n118300,A00109,CGINS-CTDMOG-10220,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00007\n3305-00101-00403\n3305-00101...,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
102,CTDMO,G,37-10221,115173\n118301,A00110,CGINS-CTDMOG-10221,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00008\n3305-00101-00426\n3305-00101...,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
103,CTDMO,G,37-10222,115174,A00111,CGINS-CTDMOG-10222,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00009\n3305-00101-00257\n3305-00101...,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMB-00004,NaN,NaN,NaN
104,CTDMO,G,37-10223,115175,A00112,CGINS-CTDMOG-10223,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00010\n3305-00101-00084\n3305-00101...,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00002\nGP03FLMB-00004,NaN,NaN,NaN


In [7]:
def get_qct_files(df, qct_directory):
    qct_dict = {}
    uids = list(set(df['UID']))
    for uid in uids:
        df['UID_match'] = df['UID'].apply(lambda x: True if uid in x else False)
        qct_series = df[df['UID_match'] == True]['QCT Testing']
        qct_series = list(qct_series.iloc[0].split('\n'))
        qct_dict.update({uid:qct_series})
    return qct_dict

In [8]:
def get_calibration_files(serial_nums, cal_directory):
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)
        sn = str(sn[:])
        files = []
        for file in os.listdir(cal_directory):
            if 'Calibration_File' in file:
                if sn in file:
                    files.append(file)
        calibration_files.update({uid:files})
    return calibration_files

In [9]:
# Now I need to load the all of the csv files based on their UID
def load_csv_info(csv_dict,filepath):
    """
    Loads the calibration coefficient information contained in asset management
    
    Args:
        csv_dict - a dictionary which associates an instrument UID to the
            calibration csv files in asset management
        filepath - the path to the directory containing the calibration csv files
    Returns:
        csv_cals - a dictionary which associates an instrument UID to a pandas
            dataframe which contains the calibration coefficients. The dataframes
            are indexed by the date of calibration
    """
    
    # Load the calibration data into pandas dataframes, which are then placed into
    # a dictionary by the UID
    csv_cals = {}
    for uid in csv_dict:
        cals = pd.DataFrame()
        for file in csv_dict[uid]:
            data = pd.read_csv(filepath+file)
            date = file.split('__')[1].split('.')[0]
            data['CAL DATE'] = pd.to_datetime(date)
            cals = cals.append(data)
        csv_cals.update({uid:cals})
        
    # Pivot the dataframe to be sorted based on calibration date
    for uid in csv_cals:
        csv_cals[uid] = csv_cals[uid].pivot(index=csv_cals[uid]['CAL DATE'], columns='name')['value']
        
    return csv_cals

In [10]:
def splitDataFrameList(df,target_column):
    ''' 
    df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
            
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df

**Identify the QCT Testing documents associated with each individual instrument (the UID)**

In [11]:
qct_dict = get_qct_files(CTDMO, qct_directory)
qct_dict

{'CGINS-CTDMOG-11535': ['3305-00101-00036',
  '3305-00101-00329',
  '3305-00101-00602'],
 'CGINS-CTDMOG-10225': ['3305-00101-00012',
  '3305-00101-00259',
  '3305-00101-00582'],
 'CGINS-CTDMOG-11639': ['3305-00101-00060',
  '3305-00101-00275',
  '3305-00101-00344',
  '3305-00101-00711'],
 'CGINS-CTDMOG-11643': ['3305-00101-00064',
  '3305-00101-00279',
  '3305-00101-00577'],
 'CGINS-CTDMOG-13439': ['3305-00101-00163',
  '3305-00101-00465',
  '3305-00101-00690'],
 'CGINS-CTDMOG-11593': ['3305-00101-00037',
  '3305-00101-00330',
  '3305-00101-00601'],
 'CGINS-CTDMOG-12588': ['3305-00101-00386', '3305-00101-00646'],
 'CGINS-CTDMOG-13635': ['3305-00101-00220',
  '3305-00101-00516',
  '3305-00101-00676'],
 'CGINS-CTDMOG-11607': ['3305-00101-00051',
  '3305-00101-00305',
  '3305-00101-00684'],
 'CGINS-CTDMOG-11648': ['3305-00101-00069',
  '3305-00101-00284',
  '3305-00101-00415',
  '3305-00101-00622'],
 'CGINS-CTDMOG-11534': ['3305-00101-00035',
  '3305-00101-00327',
  '3305-00101-00589'],
 

**Identify the calibration csvs stored in asset management which correspond to a particular instrument.**

In [12]:
csv_dict = load_asset_management(CTDMO, asset_management_directory)
csv_dict

{'CGINS-CTDMOG-10214': ['CGINS-CTDMOG-10214__20170829.csv',
  'CGINS-CTDMOG-10214__20121113.csv',
  'CGINS-CTDMOG-10214__20140214.csv',
  'CGINS-CTDMOG-10214__20151110.csv'],
 'CGINS-CTDMOG-10215': ['CGINS-CTDMOG-10215__20160915.csv',
  'CGINS-CTDMOG-10215__20121113.csv',
  'CGINS-CTDMOG-10215__20181009.csv'],
 'CGINS-CTDMOG-10216': ['CGINS-CTDMOG-10216__20170903.csv',
  'CGINS-CTDMOG-10216__20121113.csv',
  'CGINS-CTDMOG-10216__20151113.csv'],
 'CGINS-CTDMOG-10217': ['CGINS-CTDMOG-10217__20160831.csv',
  'CGINS-CTDMOG-10217__20121114.csv',
  'CGINS-CTDMOG-10217__20181009.csv'],
 'CGINS-CTDMOG-10218': ['CGINS-CTDMOG-10218__20151108.csv',
  'CGINS-CTDMOG-10218__20170901.csv',
  'CGINS-CTDMOG-10218__20121115.csv',
  'CGINS-CTDMOG-10218__20140214.csv'],
 'CGINS-CTDMOG-10219': ['CGINS-CTDMOG-10219__20160921.csv',
  'CGINS-CTDMOG-10219__20180818.csv',
  'CGINS-CTDMOG-10219__20121114.csv'],
 'CGINS-CTDMOG-10220': ['CGINS-CTDMOG-10220__20121114.csv',
  'CGINS-CTDMOG-10220__20160827.csv',
  'C

In [13]:
uids = list(set(CTDMO['UID']))

In [14]:
CTDMO['QCT Testing'] = CTDMO['QCT Testing'].apply(lambda x: x.split('\n'))

**Get the serial numbers for each CTDMO, and use those serial numbers to search for and return all of the relevant vendor documents for a particular instrument.**

In [15]:
serial_nums = get_serial_nums(CTDMO, uids)

In [16]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

{'CGINS-CTDMOG-11535': ['CTDMO-G_SBE_37IM_SN_37-11535_Calibration_Files_2014-01-17.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11535_Calibration_Files_2016-01-07.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11535_Calibration_Files_2017-09-15.zip'],
 'CGINS-CTDMOG-10225': ['CTDMO-G_SBE_37IM_SN_37-10225_Calibration_Files_2012-11-21.pdf',
  'CTDMO-G_SBE_37IM_SN_37-10225_Calibration_Files_2016-01-13.pdf',
  'CTDMO-G_SBE_37IM_SN_37-10225_Calibration_Files_2017-09-01.zip'],
 'CGINS-CTDMOG-11639': ['CTDMO-G_SBE_37IM_SN_37-11639_Calibration_Files_2014-02-05.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11639_Calibration_Files_2016-01-13.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11639_Calibration_Files_2018-08-31.zip'],
 'CGINS-CTDMOG-11643': ['CTDMO-G_SBE_37IM_SN_37-11643_Calibration_Files_2014-02-05.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11643_Calibration_Files_2016-01-13.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11643_Calibration_Files_2017-09-01.zip'],
 'CGINS-CTDMOG-13439': ['CTDMO-G_SBE_37IM_SN_37-13439_Calibration_Files_2015-06-02.pdf',
  'CTDMO-G_SBE_

In [17]:
len(uids)

164

========================================================================================================================
**Now, need to get all the files for a particular CTDMO UID:**

In [1311]:
uid = sorted(uids)[55]
uid

'CGINS-CTDMOG-11650'

In [1312]:
cal_files = sorted(cal_dict[uid])
for file in cal_files:
    print(file)

CTDMO-G_SBE_37IM_SN_37-11650_Calibration_Files_2014-02-13.pdf
CTDMO-G_SBE_37IM_SN_37-11650_Calibration_Files_2016-01-13.pdf


In [1313]:
csv_files = sorted(csv_dict[uid])
for file in csv_files:
    print(file)

CGINS-CTDMOG-11650__20140620.csv
CGINS-CTDMOG-11650__20151112.csv


In [1315]:
qct_files = sorted(qct_dict[uid])
qct_files = qct_files[0].split()
for file in qct_files:
    print(file)

3305-00101-00071
3305-00101-00286


**Get and return the full directory paths to the relevant files**

In [1316]:
csv_path = []
for cf in csv_files:
    path = generate_file_path(asset_management_directory, cf)
    csv_path.append(path)
csv_path

['/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/CTDMOG/CGINS-CTDMOG-11650__20140620.csv',
 '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/CTDMOG/CGINS-CTDMOG-11650__20151112.csv']

In [1317]:
cal_path = []
for cf in cal_files:
    path = generate_file_path(cal_directory, cf)
    cal_path.append(path)
cal_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Cal/CTDMO-G_SBE_37IM_SN_37-11650_Calibration_Files_2014-02-13.pdf',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Cal/CTDMO-G_SBE_37IM_SN_37-11650_Calibration_Files_2016-01-13.pdf']

In [1318]:
qct_path = []
for qf in qct_files:
    path = generate_file_path(qct_directory, qf)
    qct_path.append(path)
qct_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Results/3305-00101-00071-A.txt',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Results/3305-00101-00286-A.cap']

========================================================================================================================
### Processing and Parsing the raw data
With the associated vendor documents (cal files), QCT checkins (qct files), and calibration csvs (csv files), I want to be able to compare the following:
* **(1)** That the calibration date matches between the different documents
* **(2)** The file name agrees with the CTDMO UID and the calibration date
* **(3)** The calibration coefficients all agree between the different reference documents and calibration csvs
* **(4)** Identify when a calibration coefficient is incorrect, where to find it, and how to correct it

**Check that the coefficients have been parsed from the QCT into the ctdmo calibration object coefficients dictionary. There should be 24 coefficients.**

**Similarly, check the instrument serial number, the calibration date (may be more than one b/c separate calibration dates for T, S, and P sensors), and the type (for CTDMOs should be 37)**

**Now, I can load the  coefficients from the QCT file into a pandas dataframe, and use the ctd serial number and calibration date (taken as the latest of the different calibration dates) to autogenerate a csv file.**

First, I'm going to generate a temp local directory in which to save the csv. This allows for me to work locally and not accidently overwrite important existing files.

In [1319]:
temp_directory = '/'.join((os.getcwd(),'temp'))
temp_directory

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp'

In [1320]:
shutil.rmtree(temp_directory)

In [1321]:
temp_path = '/'.join((temp_directory,'qct'))
ensure_dir(temp_path)

Now I can save the file to a temporary directory under "qct"

Check that it wrote to the directory in the correct filename format

I want to do this for all of the QCT documents for a particular instrument:

In [1322]:
temp_path = '/'.join((temp_directory,'qct'))
ensure_dir(temp_path)
for qct in qct_path:
    try:
        ctdmo = CTDMOCalibration(uid=uid)
        ctdmo.mo_parse_qct(qct)
        ctdmo.write_csv(temp_path)
    except:
        print(qct)

In [1323]:
temp_directory

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp'

In [1324]:
os.listdir(temp_directory+'/qct')

['CGINS-CTDMOG-11650__20140213.csv', 'CGINS-CTDMOG-11650__20151112.csv']

========================================================================================================================
### Parse and process the vendor documents
The next step is to read and parse the vendor documents. This is a more difficult challenge, since for CTDMOs the vendor documents are retained mostly as pdf files. While the pdf files are parseable, there is an added complication in that the forms have changed over time, with sometimes the T/S/P calibration pdfs combined into a single file, whereas other times they are separated into individual files. Furthermore, the files are often zipped into a single folder. So, I have the following possible vendor documents:
* **(1)** A .cal file - this is the easiest to read and parse, in a similar format to the CTDBP .cal files
* **(2)** A combinded pdf - this is the most difficult format. Need to separate out the different pages which each separately contain either the temperature calibration info, the conductivity calibration info, or the pressure calibration info.
* **(3)** Separate pdfs - this is a simpler pdf reading schematic, where I know a priori which particular "page" will contain relevant calibration info. 

There are a couple of different pdf readers that I can use:
1. PyPDF2
2. PDFMiner
3. Textract

First, I'm going to start off by trying PyPDF2 to read the document

In [1325]:
import PyPDF2
import textract

PyPDF2 does not work to extract text from the CTDMO combined pdf file document. That means this is going to be a more complicated project to extract the relevant info. 

**When parsing the pdf file, it appears that the built-in method of pdf2text does the best job at parsing the forms, particularly the temperature and conductivity coefficients. The pressure calibration coefficients are not as well parsed, due to the positioning of the image.**

**This means that I'm going to split and use two different methods for getting the calibration coefficients depending on what the calibration is for, i.e. T/S/P. For T and S, I'll use the built-in method for extracting text. For the pressure, I'll use the tesseract OCR approach.**

========================================================================================================================
### Preprocessing the Vendor Files
In order to automate the parsing of the CTDMO calibration coefficients from pdf files into csv files that can be read by Python requires a bit of preprocessing. In particular, the following steps are taken to make parsing the files:
* **(1)** Copy or extract the vendor calibration files from the Vault location to a local temp directory
* **(2)** Iterate over the available pdfs and split multipage pdfs into single page pdfs and append _page_ to the file
* **(3)** Once the pdfs have been split, they are ready to be parsed by the CTDMO object parsers.

In [1326]:
# Now, write a function to copy over the file
def copy_to_local(cal_path):
    """
    Function which copies the files from the cal_path to a locally
    created temp directory
    """
    
    for filepath in cal_path:
        # Create a folder in which to save extracted data
        folder, *ignore = filepath.split('/')[-1].split('.')
        savedir = '/'.join((os.getcwd(),'temp','cal_data',folder))
        # Now make sure that the save directory exists and can be used
        ensure_dir(savedir)
    
        if filepath.endswith('.zip'):
            with ZipFile(filepath,'r') as zfile:
                for file in zfile.namelist():
                    zfile.extract(file,path=savedir)    
        else:
            shutil.copy(filepath, savedir)

In [1327]:
copy_to_local(cal_path)

In [1328]:
def split_multipage_pdfs(caldir):
    
    folders = os.listdir(caldir)
    for folder in folders:
        filepath = '/'.join((os.getcwd(),'temp','cal_data',folder))
    
        if any([file for file in os.listdir(filepath) if file.endswith('.cal')]):
            pass
        
        else:
            files = [file for file in os.listdir(filepath) if 'SERVICE REPORT' not in file]
        
            for file in files:
                inputpath = '/'.join((filepath,file))
                inputpdf = PyPDF2.PdfFileReader(inputpath, 'rb')
            
                for i in range(inputpdf.numPages):
                    output = PyPDF2.PdfFileWriter()
                    output.addPage(inputpdf.getPage(i))
                    filename = '_'.join((inputpath.split('.')[0], 'page', str(i)))
                    with open(filename+'.pdf', "wb") as outputStream:
                        output.write(outputStream)

In [1329]:
split_multipage_pdfs('/'.join((os.getcwd(),'temp','cal_data')))

**The next step is to iterate over the vendor calibration files and extract the calibration coefficients from the files. This is done by starting an instance of the CTDMO calibration object, check if any of the calibration data is stored as a .cal file, if no .cal file loop over the other files looking for _page_ files which indicates that the pdf file has been prepped.**

In [1330]:
datadir = os.path.abspath('/'.join((os.getcwd(),'temp','cal_data')))
for folder in os.listdir(datadir):
    # Okay, now start generating calibration csvs
    ctdmo = CTDMOCalibration(uid)
    files = [file for file in os.listdir('/'.join((datadir,folder)))]
    if any([file for file in files if file.endswith('.cal')]):
        for file in files:
            if file.endswith('.cal'):
                ctdmo.mo_parse_cal('/'.join((datadir,folder,file)))
    else:
        for file in files:
            if '_page_' in file:
                ctdmo.mo_parse_pdf('/'.join((datadir,folder,file)))
    savedir = '/'.join((os.getcwd(),'temp','cal'))
    ensure_dir(savedir)
    ctdmo.write_csv(savedir)

**Check that the calibration object properly loaded all of the calibration coefficients, serial number, calibration date, etc., and wrote the appropriate csv file.**

In [1331]:
os.listdir(temp_directory+'/cal')

['CGINS-CTDMOG-11650__20140213.csv', 'CGINS-CTDMOG-11650__20151112.csv']

In [1332]:
ctdmo.coefficients

{'CC_g': '-9.868955e-001',
 'CC_h': '1.465695e-001',
 'CC_i': '-5.026112e-004',
 'CC_j': '5.873030e-005',
 'CC_cpcor': '-9.5700e-008',
 'CC_ctcor': '3.2500e-006',
 'CC_wbotc': '7.3633e-007',
 'CC_p_range': '1450',
 'CC_pa0': '5.243975e-002',
 'CC_ptca0': '5.247241e+005',
 'CC_pa1': '4.464722e-003',
 'CC_ptca1': '-1.693212e+000',
 'CC_pa2': '-1.871982e-011',
 'CC_ptca2': '1.107708e-001',
 'CC_ptempa0': '-7.026161e+001',
 'CC_ptcb0': '2.524263e+001',
 'CC_ptempa1': '5.031098e-002',
 'CC_ptcb1': '1.250000e-004',
 'CC_ptempa2': '-4.028384e-007',
 'CC_ptcb2': '0.000000e+000',
 'CC_a0': '-1.310470e-004',
 'CC_a1': '3.099306e-004',
 'CC_a2': '-4.593363e-006',
 'CC_a3': '2.063279e-007'}

In [1333]:
len(ctdmo.coefficients)

24

In [1334]:
ctdmo.serial

'11650'

In [1335]:
ctdmo.ctd_type

'37'

In [1336]:
ctdmo.date

{0: '20140213', 1: '20140205', 2: '20140213'}

========================================================================================================================
### Compare results
Now, with QCT files parsed into csvs which follow the UFrame format, I can load both the QCT and the calibratoin csvs into pandas dataframes, which will allow element by element comparison in relatively few lines of code.

In [1337]:
def get_file_date(x):
    x = str(x)
    ind1 = x.index('__')
    ind2 = x.index('.')
    return x[ind1+2:ind2]

**Load the calibration csvs:**

In [1338]:
# Now we want to compare dataframe
csv_files = pd.DataFrame(sorted(csv_dict[uid]),columns=['csv'])
csv_files['cal date'] = csv_files['csv'].apply(lambda x: get_file_date(x))
csv_files.set_index('cal date',inplace=True)
csv_files

,csv
cal date,
20140620,CGINS-CTDMOG-11650__20140620.csv
20151112,CGINS-CTDMOG-11650__20151112.csv


In [1339]:
# Put the csv files into a similar temp directory for local working
for file in csv_dict[uid]:
    csv_savepath = '/'.join((os.getcwd(),'temp','csv'))
    ensure_dir('/'.join((os.getcwd(),'temp','csv')))
    # Now save the csv into the temp directory
    shutil.copy('/'.join((asset_management_directory,file)), csv_savepath)

In [1340]:
os.listdir(temp_directory+'/csv')

['CGINS-CTDMOG-11650__20140620.csv', 'CGINS-CTDMOG-11650__20151112.csv']

**Load the QCT csvs:**

In [1341]:
# Now we want to compare dataframe
qct_files = pd.DataFrame(sorted(os.listdir('temp/qct')),columns=['qct'])
qct_files['cal date'] = qct_files['qct'].apply(lambda x: get_file_date(x))
qct_files.set_index('cal date',inplace=True)
qct_files

,qct
cal date,
20140213,CGINS-CTDMOG-11650__20140213.csv
20151112,CGINS-CTDMOG-11650__20151112.csv


**Load the calibration csvs:**

In [1342]:
cal_files = pd.DataFrame(sorted(os.listdir('temp/cal')),columns=['cal'])
cal_files['cal date'] = cal_files['cal'].apply(lambda x: get_file_date(x))
cal_files.set_index('cal date',inplace=True)
cal_files

,cal
cal date,
20140213,CGINS-CTDMOG-11650__20140213.csv
20151112,CGINS-CTDMOG-11650__20151112.csv


**Combine the dataframes into one in order to know which csv files to compare and check calibration dates.**

In [1350]:
df_files = csv_files.join(qct_files,how='outer').join(cal_files,how='outer').fillna(value='-999')
df_files

,csv,qct,cal
cal date,,,
20140213,CGINS-CTDMOG-11650__20140213.csv,CGINS-CTDMOG-11650__20140213.csv,CGINS-CTDMOG-11650__20140213.csv
20151112,CGINS-CTDMOG-11650__20151112.csv,CGINS-CTDMOG-11650__20151112.csv,CGINS-CTDMOG-11650__20151112.csv


In [1344]:
df_files['csv']

cal date
20140213                                -999
20140620    CGINS-CTDMOG-11650__20140620.csv
20151112    CGINS-CTDMOG-11650__20151112.csv
Name: csv, dtype: object

**If the filename is wrong, the calibration coefficient checker will not manage to compare the results. Consequently, we'll make a local copy of the wrong file to a new file with the correct name, and then run the calibration coefficient checker.**

In [1345]:
a = 'temp/csv/' + 'CGINS-CTDMOG-11650__20140620.csv'
b = 'temp/csv/' + 'CGINS-CTDMOG-11650__20140213.csv'

In [1346]:
shutil.copy(a,b)

'temp/csv/CGINS-CTDMOG-11650__20140213.csv'

In [1347]:
os.listdir('temp/csv')

['CGINS-CTDMOG-11650__20140620.csv',
 'CGINS-CTDMOG-11650__20140213.csv',
 'CGINS-CTDMOG-11650__20151112.csv']

In [1348]:
!rm 'temp/csv/CGINS-CTDMOG-11650__20140620.csv'

In [1349]:
csv_files = pd.DataFrame(sorted(os.listdir('temp/csv')),columns=['csv'])
csv_files['cal date'] = csv_files['csv'].apply(lambda x: get_file_date(x))
csv_files.set_index('cal date',inplace=True)
csv_files

,csv
cal date,
20140213,CGINS-CTDMOG-11650__20140213.csv
20151112,CGINS-CTDMOG-11650__20151112.csv


**Update the pandas dataframe with the new renamed csv**

**This table tells us that, for the csv CGINS-CTDMOG-11596__20150608.csv, I am missing a QCT document and vendor doc which could verify the calibration coefficients. Next, for the files I can compare, I want to go through and check each calibration coefficient.**

In [1083]:
def check_cal_coeffs(coeffs_dict):
    
    # Part 1: coeff by coeff comparison between each source of coefficients
    keys = list(coeffs_dict.keys())
    comparison = {}
    for i in range(len(keys)):
        names = (keys[i], keys[i - (len(keys)-1)])
        check = len(coeffs_dict.get(keys[i])['value']) == len(coeffs_dict.get(keys[i - (len(keys)-1)])['value'])
        if check:
            compare = np.isclose(coeffs_dict.get(keys[i])['value'], coeffs_dict.get(keys[i - (len(keys)-1)])['value'])
            comparison.update({names:compare})
        else:
            pass
        
    # Part 2: now do a logical_and comparison between the results from part 1
    keys = list(comparison.keys())
    i = 0
    mask = comparison.get(keys[i])
    while i < len(keys)-1:
        i = i + 1
        mask = np.logical_and(mask, comparison.get(keys[i]))
        print(i)
       
    return mask 

In [1351]:
result = {}
for cal_date in df_files.index:
    # Part 1, load all of the csv files
    coeffs_dict = {}
    for source,fname in df_files.loc[cal_date].items():
        if fname != '-999':
            load_directory = '/'.join((os.getcwd(),'temp',source,fname))
            df_coeffs = pd.read_csv(load_directory)
            for i in list(set(df_coeffs['serial'])):
                print(source + '-' + fname + ': ' + str(i))
            df_coeffs.set_index(keys='name',inplace=True)
            df_coeffs.sort_index(inplace=True)
            coeffs_dict.update({source:df_coeffs})
        else:
            pass
    
    # Part 2, now check the calibration coefficients
    mask = check_cal_coeffs(coeffs_dict)
    
    # Part 3: get the calibration coefficients are wrong
    # and show them
    fname = df_files.loc[cal_date]['csv']
    if fname == '-999':
        incorrect = 'No csv file.'
    else:
        incorrect = coeffs_dict['csv'][mask == False]
    result.update({fname:incorrect})

csv-CGINS-CTDMOG-11650__20140213.csv: 37-11650
qct-CGINS-CTDMOG-11650__20140213.csv: 37-11650
cal-CGINS-CTDMOG-11650__20140213.csv: 37-11650
1
2
csv-CGINS-CTDMOG-11650__20151112.csv: 37-11650
qct-CGINS-CTDMOG-11650__20151112.csv: 37-11650
cal-CGINS-CTDMOG-11650__20151112.csv: 37-11650
1
2


In [1352]:
result

{'CGINS-CTDMOG-11650__20140213.csv': Empty DataFrame
 Columns: [serial, value, notes]
 Index: [], 'CGINS-CTDMOG-11650__20151112.csv':         serial     value  notes
 name                           
 CC_i  37-11650 -0.000458    NaN}

In [ ]:
filepath = temp_directory+'/cal_data/CTDMO-G_SBE_37IM_SN_37-10258_Calibration_Files_2012-12-06/CTDMO-G_SBE_37IM_SN_37-10258_Calibration_Files_2012-12-06_page_3.pdf'

In [ ]:
ptext = textract.process(filepath, method='tesseract', encoding='utf-8')
ptext = ptext.replace(b'\xe2\x80\x94',b'-')
ptext = ptext.decode('utf-8')

In [ ]:
for line in ptext.splitlines():
    print(line)

In [ ]:
for line in ptext.splitlines():
    if '-' in line:
        if 'PAIL' in line:
            stuff = line.split()[-1]
            print(stuff)

In [ ]:
stuff[0].encode('utf-8')

In [ ]:
'-'.encode('utf-8')

In [ ]:
stuff.replace(b'\xe2\x80\x94',b'-')

In [ ]:
print(ptext)

In [39]:
import datetime
import re
import os
from wcmatch import fnmatch
import pandas as pd
import numpy as np
import string
from zipfile import ZipFile
import json
import PyPDF2
import textract

class CTDMOCalibration():
    # Class that stores calibration values for CTDs.

    def __init__(self, uid):
        self.serial = ''
        self.uid = uid
        self.ctd_type = uid
        self.coefficients = {}
        self.date = {}

        # Name mapping for the MO-type CTDs (when reading from pdfs)
        self.mo_coefficient_name_map = {
            'ptcb1': 'CC_ptcb1',
            'pa2': 'CC_pa2',
            'a3': 'CC_a3',
            'pa0': 'CC_pa0',
            'wbotc': 'CC_wbotc',
            'ptcb0': 'CC_ptcb0',
            'g': 'CC_g',
            'ptempa1': 'CC_ptempa1',
            'ptcb2': 'CC_ptcb2',
            'a0': 'CC_a0',
            'h': 'CC_h',
            'ptca0': 'CC_ptca0',
            'a2': 'CC_a2',
            'cpcor': 'CC_cpcor',
            'pcor':'CC_cpcor',
            'i': 'CC_i',
            'ptempa0': 'CC_ptempa0',
            'prange': 'CC_p_range',
            'ctcor': 'CC_ctcor',
            'tcor':'CC_ctcor',
            'a1': 'CC_a1',
            'j': 'CC_j',
            'ptempa2': 'CC_ptempa2',
            'pa1': 'CC_pa1',
            'ptca1': 'CC_ptca1',
            'ptca2': 'CC_ptca2',
        }

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = d.split('-')[2]
            self._uid = d
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")

    @property
    def ctd_type(self):
        return self._ctd_type

    @ctd_type.setter
    def ctd_type(self, d):
        if 'MO' in d:
            self._ctd_type = '37'
        elif 'BP' in d:
            self._ctd_type = '16'
        else:
            self._ctd_type = ''

            
    def mo_parse_pdf(self, filepath):
        """
        This function extracts the text from a given pdf file.
        Depending on if the text concerns calibration for 
        temperature/conductivity or pressure, it calls a further
        function to parse out the individual calibration coeffs.
    
        Args:
            filepath - the full directory path to the pdf file
                which it to be extracted and parsed.
        Calls:
            mo_parse_p(text, filepath)
            mo_parse_ts(text)
        Returns:
            self - a CTDMO calibration object with calibration
                coefficients parsed into the object calibration
                dictionary
        """
    
        text = textract.process(filepath, encoding='utf-8')
        text = text.decode('utf-8')
    
        if 'PRESSURE CALIBRATION DATA' in text:
            self.mo_parse_p(filepath)
    
        elif 'TEMPERATURE CALIBRATION DATA' or 'CONDUCTIVITY CALIBRATION DATA' in text:
            self.mo_parse_ts(text)
        
        else:
            pass
    

    def mo_parse_ts(self, text):
        """
        This function parses text from a pdf and loads the appropriate calibration
        coefficients for the temperature and conductivity sensors into the CTDMO 
        calibration object.
    
        Args:
            text - extracted text from a pdf page
        Returns:
            self - a CTDMO calibration object with either temperature or conductivity
                calibration values filled in the calibration coefficients dictionary
        Raises:
            Exception - if the serial number in the pdf text does not match the
                serial number parsed from the UID
        """
    
        keys = self.mo_coefficient_name_map.keys()
        for line in text.splitlines():
    
            if 'CALIBRATION DATE' in line:
                *ignore, cal_date = line.split(':')
                cal_date = pd.to_datetime(cal_date).strftime('%Y%m%d')
                self.date.update({len(self.date): cal_date})
        
            elif 'SERIAL NUMBER' in line:
                *ignore, serial_num = line.split(':')
                serial_num = serial_num.strip()
                if serial_num != self.serial:
                    raise Exception(f'Instrument serial number {serial_num} does not match UID serial num {self.serial}')
           
            elif '=' in line:
                key, *ignore, value = line.split()
                name = self.mo_coefficient_name_map.get(key.strip().lower())
                if name is not None:
                    self.coefficients.update({name: value.strip()})
            else:
                continue
            
            
    def mo_parse_p(self,filepath):
        """
        Function to parse the pressure calibration information from a pdf. To parse
        the pressure cal info requires re-extracting the text from the pdf file using
        tesseract-ocr rather than the basic pdf2text converter.
    
        Args:
            text - extracted text from a pdf page using pdf2text
            filepath - full directory path to the pdf file containing the pressure
                calibration info. This is the file which will be re-extracted.
        Returns
            self - a CTDMO calibration object with pressure calibration values filled
                in the calibration coefficients dictionary
        """
    
        # Now, can reprocess using tesseract-ocr rather than pdftotext
        ptext = textract.process(filepath, method='tesseract', encoding='utf-8')
        ptext = ptext.replace(b'\xe2\x80\x94',b'-')
        ptext = ptext.decode('utf-8')
        keys = list(self.mo_coefficient_name_map.keys())
        
        # Get the calibration date:
        for line in ptext.splitlines():
            if 'CALIBRATION DATE' in line:
                items = line.split()
                ind = items.index('DATE:')
                cal_date = items[ind+1]
                cal_date = pd.to_datetime(cal_date).strftime('%Y%m%d')
                self.date.update({len(self.date):cal_date})
            
            if 'psia S/N' in line:
                items = line.split()
                ind = items.index('psia')
                prange = items[ind-1]
                name = self.mo_coefficient_name_map.get('prange')
                self.coefficients.update({name: prange})
    
            # Loop through each line looking for the lines which contain
            # calibration coefficients
            if '=' in line:
                # Tesseract-ocr misreads '0' as O, and 1 as IL
                line = line.replace('O','0').replace('IL','1').replace('=','').replace(',.','.').replace(',','.')
                line = line.replace('L','1').replace('@','0').replace('l','1').replace('--','-')
                if '11' in line and 'PA2' not in line:
                    line = line.replace('11','1')
                items = line.split()
                for n, k in enumerate(items):
                    if k.lower() in keys:
                        try:
                            float(items[n+1])
                            name = self.mo_coefficient_name_map.get(k.lower())
                            self.coefficients.update({name: items[n+1]})
                        except:
                            pass
        if 'CC_ptcb2' not in list(self.mo_coefficient_name_map.keys()):
            self.coefficients.update({'CC_ptcb2': '0.000000e+000'})


    def mo_parse_cal(self, filepath):
        """
        Function to parse the .cal file for the CTDMO when a .cal file
        is available.
        """
    
        if not filepath.endswith('.cal'):
            raise Exception(f'Not a .cal filetype.')
    
        with open(filepath) as file:
            data = file.read()
        
        for line in data.splitlines():
            key, value = line.split('=')
            key = key.strip()
            value = value.strip()
        
            if 'SERIALNO' in key:
                sn = value
                if self.serial != sn:
                    raise Exception(f'File serial number {sn} does not match UID {self.uid}')
                
            elif 'CALDATE' in key:
                cal_date = pd.to_datetime(value).strftime('%Y%m%d')
                self.date.update({len(self.date): cal_date})
            
            elif 'INSTRUMENT_TYPE' in key:
                ctd_type = value[-2:]
                if self.ctd_type != ctd_type:
                    raise Exception(f'CTD type {ctd_type} does not match uid {self.uid}.')
                
            else:
                if key.startswith('T'):
                    key = key.replace('T','')
                if key.startswith('C') and len(key)==2:
                    key = key.replace('C','')
                name = self.mo_coefficient_name_map.get(key.lower())
                if name is not None:
                    self.coefficients.update({name: value})
                    
        # Now we need to add in the range of the sensor
        name = self.mo_coefficient_name_map.get('prange')
        self.coefficients.update({name: '1450'})

                    
    def mo_parse_qct(self, filepath):
        """
        This function reads and parses the QCT file into
        the CTDMO calibration object.
    
        Args:
            filepath - full directory path and filename of
                the QCT file
        Returns:
        
        """
        
        with open(filepath,errors='ignore') as file:
            data = file.read()

        data = data.replace('<',' ').replace('>',' ')
        keys = self.mo_coefficient_name_map.keys()

        for line in data.splitlines():
            items = line.split()
    
            # If the line is empty, go to next line
            if len(items) == 0:
                continue
    
            # Check the serial number from the instrument
            elif 'SERIAL NO' in line:
                ind = items.index('NO.')
                sn = items[ind+1]
                if sn != self.serial:
                    raise Exception(f'Serial number {sn} in QCT document does not match uid serial number {self.serial}')
        
            # Check if the line contains the calibration date
            elif 'CalDate' in line:
                cal_date = pd.to_datetime(items[1]).strftime('%Y%m%d')
                self.date.update({len(self.date): cal_date})
        
            # Get the coefficient names and values
            elif items[0].lower() in keys:
                name = self.mo_coefficient_name_map[items[0].lower()]
                self.coefficients.update({name: items[1]})
        
            else:
                pass
    
    
    def write_csv(self, outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.

        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is
                written to the specified directory from the outpath.
        """

        # Run a check that the coefficients have actually been loaded
        if len(self.coefficients) == 0:
            raise ValueError('No calibration coefficients have been loaded.')

        # Create a dataframe to write to the csv
        data = {'serial': [self.ctd_type + '-' + self.serial]*len(self.coefficients),
                'name': list(self.coefficients.keys()),
                'value': list(self.coefficients.values()),
                'notes': ['']*len(self.coefficients)
                }
        df = pd.DataFrame().from_dict(data)

        # Generate the csv name
        cal_date = max(self.date.values())
        csv_name = self.uid + '__' + cal_date + '.csv'

        # Write the dataframe to a csv file
        # check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        check = 'y'
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)

In [ ]:
import datetime
import re
import os
from wcmatch import fnmatch
import pandas as pd
import numpy as np
import string
from zipfile import ZipFile
import json

class CTDMOCalibration():
    # Class that stores calibration values for CTDs.

    def __init__(self, uid):
        self.serial = ''
        self.uid = uid
        self.ctd_type = uid
        self.coefficients = {}
        self.date = {}

        # Name mapping for the MO-type CTDs (when reading from pdfs)
        self.mo_coefficient_name_map = {
            'ptcb1': 'CC_ptcb1',
            'pa2': 'CC_pa2',
            'a3': 'CC_a3',
            'pa0': 'CC_pa0',
            'wbotc': 'CC_wbotc',
            'ptcb0': 'CC_ptcb0',
            'g': 'CC_g',
            'ptempa1': 'CC_ptempa1',
            'ptcb2': 'CC_ptcb2',
            'a0': 'CC_a0',
            'h': 'CC_h',
            'ptca0': 'CC_ptca0',
            'a2': 'CC_a2',
            'cpcor': 'CC_cpcor',
            'pcor':'CC_cpcor',
            'i': 'CC_i',
            'ptempa0': 'CC_ptempa0',
            'prange': 'CC_p_range',
            'ctcor': 'CC_ctcor',
            'tcor':'CC_ctcor',
            'a1': 'CC_a1',
            'j': 'CC_j',
            'ptempa2': 'CC_ptempa2',
            'pa1': 'CC_pa1',
            'ptca1': 'CC_ptca1',
            'ptca2': 'CC_ptca2',
        }

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = d.split('-')[2]
            self._uid = d
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")

    @property
    def ctd_type(self):
        return self._ctd_type

    @ctd_type.setter
    def ctd_type(self, d):
        if 'MO' in d:
            self._ctd_type = '37'
        elif 'BP' in d:
            self._ctd_type = '16'
        else:
            self._ctd_type = ''
            
            

    def mo_parse_pdf(self, temp_path):
        text = {}
        for file in os.listdir(temp_path):
            # Read the pdf using OCR
            filepath = '/'.join((temp_path, file))
            pdf = textract.process(filepath, method='pdfminer', language='eng', encoding='ascii')
            # Now convert the extracted pdf text to ASCII
            pdf = pdf.decode('ascii')
            # Save the text in the dictionary
            text.update({file: pdf})
        
        return text


    def mo_parse_pressure_cal(self,text):
    
        values = []
        coeffs = []
        flag = False
        for item in text.split('\n'):
    
            if 'COEFFICIENTS' in item:
                flag = True
                continue
    
            if 'SPAN' in item and flag is True:
                flag = False
        
            if 'S/N' in item:
                prange = item.split()[0]
                coeffs.append('prange')
                values.append(prange)
        
            if flag is True and len(item) > 0:
                data = item.split()
                try:
                    float(data[0])
                    values.append(data[0].strip())
                except:
                    coeffs.append(data[0].strip())
            
        for n, k in enumerate(coeffs):
            name = self.mo_coefficient_name_map[k.lower()]
            self.coefficients.update({name: values[n]})
        
        
    def mo_parse_ts_cal(self,text):
        keys = list(self.mo_coefficient_name_map.keys())

        for item in text.split('\n'):
    
            # Get the serial number
            if 'SERIAL NUMBER' in item:
                *ignore, sn = item.split(':')
        
            # Get the calibration date    
            if 'CALIBRATION DATE' in item:
                *ignore, cal_date = item.split(':')
                cal_date = pd.to_datetime(cal_date).strftime('%Y%m%d')
                self.date.update({len(self.date):cal_date})
        
            # Get the coefficient names and values
            if any([word for word in item.strip().split() if word.lower() in keys]):
                # Eliminate the returned lines that are equations
                try:
                    coeff, value = item.split('=')
                    float(value)
                    print(coeff, value)
                    name = self.mo_coefficient_name_map[coeff.lower().strip()]
                    self.coefficients.update({name: value.strip()})
                except:
                    pass

    
    def mo_load_pdf(self,temp_path):
    
        text = parse_pdf(temp_path)
    
        for k in text.keys():
            if 'P'+self.serial in k:
                parse_pressure_cal(text[k])
            
            else:
                parse_ts_cal(text[k])
            
    
    def mo_parse_qct(self,filepath):
        """
        This function reads and parses the QCT file into
        the CTDMO calibration object.
    
        Args:
            filepath - full directory path and filename of
                the QCT file
        Returns:
        
        """
        
        with open(filepath,errors='ignore') as file:
            data = file.read()

        data = data.replace('<',' ').replace('>',' ')
        keys = self.mo_coefficient_name_map.keys()

        for line in data.splitlines():
            items = line.split()
    
            # If the line is empty, go to next line
            if len(items) == 0:
                continue
    
            # Check the serial number from the instrument
            elif 'SERIAL NO' in line:
                ind = items.index('NO.')
                sn = items[ind+1]
                if sn != self.serial:
                    raise Exception(f'Serial number {sn} in QCT document does not match uid serial number {self.serial}')
        
            # Check if the line contains the calibration date
            elif 'CalDate' in line:
                cal_date = pd.to_datetime(items[1]).strftime('%Y%m%d')
                self.date.update({len(self.date): cal_date})
        
            # Get the coefficient names and values
            elif items[0].lower() in keys:
                name = self.mo_coefficient_name_map[items[0].lower()]
                self.coefficients.update({name: items[1]})
        
            else:
                pass
    
    
    def write_csv(self,outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.

        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is
                written to the specified directory from the outpath.
        """

        # Run a check that the coefficients have actually been loaded
        if len(self.coefficients) == 0:
            raise ValueError('No calibration coefficients have been loaded.')

        # Create a dataframe to write to the csv
        data = {'serial': [self.ctd_type + '-' + self.serial]*len(self.coefficients),
                'name': list(self.coefficients.keys()),
                'value': list(self.coefficients.values()),
                'notes': ['']*len(self.coefficients)
                }
        df = pd.DataFrame().from_dict(data)

        # Generate the csv name
        cal_date = max(self.date.values())
        csv_name = self.uid + '__' + cal_date + '.csv'

        # Write the dataframe to a csv file
        # check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        check = 'y'
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)

In [ ]:
for i in list(CSV.index):
    if i not in CTD.coefficients.keys():
        print(i)

In [ ]:
# Try using textract to get the pdf text
import textract
import nltk

In [ ]:
temp_path = '/'.join((os.getcwd(),'temp','qct'))
ensure_dir(temp_path)
for file in qct_files:
    qct_path = generate_file_path(qct_directory, file)
    CTD = CTDCalibration(uid=uid)
    CTD.mo_read_qct(qct_path)
    CTD.write_csv(temp_path)

In [ ]:
os.listdir(temp_path)

In [ ]:
text = {}
for file in os.listdir(temp_path):
    # Read the pdf using OCR
    filepath = '/'.join((temp_path, file))
    pdf = textract.process(filepath, method='pdfminer', language='eng', encoding='ascii')
    # Now convert the extracted pdf text to ASCII
    pdf = pdf.decode('ascii')
    # Save the text in the dictionary
    text.update({file: pdf})
    

In [ ]:
# This parses the Pressure calibration document, which needs to be handled differently than the conductivity
# and temperature calibration documents
values = []
coeffs = []
flag = False
for item in text['SBE 37 V2 P10216 28Aug17.pdf'].split('\n'):
    
    if 'COEFFICIENTS' in item:
        flag = True
        continue
    
    if 'SPAN' in item and flag is True:
        flag = False
        
    if 'S/N' in item:
        prange = item.split()[0]
        coeffs.append('prange')
        values.append(prange)
        
    if flag is True and len(item) > 0:
        data = item.split()
        try:
            float(data[0])
            values.append(data[0].strip())
        except:
            coeffs.append(data[0].strip())
            
for n, k in enumerate(coeffs):
    name = CTD.mo_coefficient_name_map[k.lower()]
    CTD.coefficients.update({name: values[n]})
    
CTD.coefficients

In [ ]:
def generate_file_path(dirpath, filename, ext=['.cap', '.txt', '.log'], exclude=['_V', '_Data_Workshop']):
    """
    Function which searches for the location of the given file and returns
    the full path to the file.

    Args:
        dirpath - parent directory path under which to search
        filename - the name of the file to search for
        ext - file endings to search for
        exclude - optional list which allows for excluding certain
            directories from the search
    Returns:
        fpath - the file path to the filename from the current
            working directory.
    """
    # Check if the input file name has an extension already
    # If it does, parse it for input into the search algo
    if '.' in filename:
        check = filename.split('.')
        filename = check[0]
        ext = ['.'+check[1]]

    for root, dirs, files in os.walk(dirpath):
        dirs[:] = [d for d in dirs if d not in exclude]
        for fname in files:
            if fnmatch.fnmatch(fname, [filename+'*'+x for x in ext]):
                fpath = os.path.join(root, fname)
                return fpath